In [3]:
#!pip install -U dhlab
import dhlab as dh

from dhlab import Corpus
from dhlab.api.dhlab_api import document_corpus
import requests
from sentiment import *
import pandas as pd

from dhlab.api.dhlab_api import urn_collocation, get_document_frequencies, collocation, word_variant, word_paradigm
from dhlab.text.utils import urnlist




wordcloud er ikke installert, kan ikke lage ordskyer


# Sentimentanalyse i aviskorpus 

1. [Hent inn korpus](#hent-inn-korpusdefinisjon)
2. [Tell ordfrekvens av nøkkelord](#forekomster-per-avisutgave)
3. [Score sentiment for forekomster](#sentimentscore) 
4. [Lagre utdata](#lagre-data)

## Korpusdefinisjon fra fil

**Fyll inn filsti** til lokal csv- eller excel-fil med korpusdefinisjon:


In [ ]:
# INPUT KREVES
file_path = "FYLL INN"  

In [4]:
#FIXME: slett celle

file_path = "./digavis-1999-2022.xlsx"


In [5]:
# korpusdefinisjonen tas vare på i corpus-variabelen som et dhlab.Corpus-objekt

if file_path.endswith(".xlsx"):
    corpus = dh.Corpus.from_df(pd.read_excel(file_path)) 
elif file_path.endswith(".csv"):
    corpus = dh.Corpus.from_csv(file_path)
else:
    print("file_path must be a .csv or .xlsx file: ", file_path)

In [56]:
corpus.frame

,dhlabid,urn,title,authors,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype
0,201739907,URN:NBN:no-nb_digavis_tonsbergsblad_null_null_...,tonsbergsblad,NaN,NaN,NaN,NaN,Tønsberg,20010612,2001,NaN,NaN,NaN,NaN,NaN,NaN,digavis
1,200383127,URN:NBN:no-nb_digavis_dagsavisen_null_null_200...,dagsavisen,NaN,NaN,NaN,NaN,Oslo,20040521,2004,NaN,NaN,NaN,NaN,NaN,NaN,digavis
2,202575617,URN:NBN:no-nb_digavis_nordhordland_null_null_2...,nordhordland,NaN,NaN,NaN,NaN,Alver,20160604,2016,NaN,NaN,NaN,NaN,NaN,NaN,digavis
3,204021898,URN:NBN:no-nb_digavis_vesteraalen_null_null_20...,vesteraalen,NaN,NaN,NaN,NaN,NaN,20071228,2007,NaN,NaN,NaN,NaN,NaN,NaN,digavis
4,201162018,URN:NBN:no-nb_digavis_nordhordland_null_null_2...,nordhordland,NaN,NaN,NaN,NaN,Alver,20200122,2020,NaN,NaN,NaN,NaN,NaN,NaN,digavis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,200201844,URN:NBN:no-nb_digavis_bergensavisen_null_null_...,bergensavisen,NaN,NaN,NaN,NaN,Bergen,20130720,2013,NaN,NaN,NaN,NaN,NaN,NaN,digavis
19996,203141526,URN:NBN:no-nb_digavis_lindesnes_null_null_2008...,lindesnes,NaN,NaN,NaN,NaN,Mandal,20081023,2008,NaN,NaN,NaN,NaN,NaN,NaN,digavis
19997,203037100,URN:NBN:no-nb_digavis_askoyvaeringen_null_null...,askoyvaeringen,NaN,NaN,NaN,NaN,Askøy,20081021,2008,NaN,NaN,NaN,NaN,NaN,NaN,digavis
19998,201161506,URN:NBN:no-nb_digavis_nordhordland_null_null_2...,nordhordland,NaN,NaN,NaN,NaN,Alver,20131023,2013,NaN,NaN,NaN,NaN,NaN,NaN,digavis


In [57]:
# FIXME: flytt kode inn i hjelpemodul
def strip_empty_cols(dhobj): 
    return dhobj.frame.dropna(axis=1, how="all").fillna("")

In [67]:
# Fjern kolonner som ikke har data i seg 
news_df = strip_empty_cols(corpus)

In [61]:
news_df

,dhlabid,urn,title,city,timestamp,year,doctype
0,201739907,URN:NBN:no-nb_digavis_tonsbergsblad_null_null_...,tonsbergsblad,Tønsberg,20010612,2001,digavis
1,200383127,URN:NBN:no-nb_digavis_dagsavisen_null_null_200...,dagsavisen,Oslo,20040521,2004,digavis
2,202575617,URN:NBN:no-nb_digavis_nordhordland_null_null_2...,nordhordland,Alver,20160604,2016,digavis
3,204021898,URN:NBN:no-nb_digavis_vesteraalen_null_null_20...,vesteraalen,,20071228,2007,digavis
4,201162018,URN:NBN:no-nb_digavis_nordhordland_null_null_2...,nordhordland,Alver,20200122,2020,digavis
...,...,...,...,...,...,...,...
19995,200201844,URN:NBN:no-nb_digavis_bergensavisen_null_null_...,bergensavisen,Bergen,20130720,2013,digavis
19996,203141526,URN:NBN:no-nb_digavis_lindesnes_null_null_2008...,lindesnes,Mandal,20081023,2008,digavis
19997,203037100,URN:NBN:no-nb_digavis_askoyvaeringen_null_null...,askoyvaeringen,Askøy,20081021,2008,digavis
19998,201161506,URN:NBN:no-nb_digavis_nordhordland_null_null_2...,nordhordland,Alver,20131023,2013,digavis


## Ordfrekvens av nøkkelord

**Tell ordfrekvensen** til nøkkelordet eller flere ord per utgivelse.
  
Fyll inn listen med søkeord etter eget ønske.

In [69]:
# INPUT KREVES
search_terms = """
barnevern
barnevernet
barnevernets
barneverntjeneste
barneverntjenesten
barneverntjenester
barnevernloven
barnevernsnemda
"""

In [70]:
search_terms = make_list(search_terms)          # Gjør om flerlinje-strengen til en liste
count_terms = corpus.count(search_terms)        # Tell opp forekomstene av søkeordene i korpuset
word_freqs = count_terms.frame.T                # Snu om på rader/kolonner

### Forekomster per avisutgave

`word_freqs` viser absolutt frekvens for hvert søkeord (`word`) per avisutgivelse (`urn`).

Begrepsforklaring:
- `urn`: Unique resource name. Nasjonalbibliotekets interne ID til det scannede bildet av et papirdokument. 
  - Scanningsdatoen er bakt inn i URN-en i ISO-8601-format, og samsvarer som regel med utgivelsesdatoen.
  - Avistittelen er også bakt inn i URN-en
  - Eksempel: `URN:NBN:no-nb_digavis_aamliavisa_null_null_20090319_3_11_1` er Åmliavisas avisutgave den 19.mars 2009
- `word`: eksakt ordform som forekommer i teksten.

In [71]:
word_freqs

word,barnevern,barnevernet,barnevernets,barnevernloven,barnevernsnemda,barneverntjeneste,barneverntjenesten,barneverntjenester
urn,,,,,,,,
URN:NBN:no-nb_digavis_aamliavisa_null_null_20090319_3_11_1,1.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
URN:NBN:no-nb_digavis_aamliavisa_null_null_20100114_3_1_1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
URN:NBN:no-nb_digavis_aamliavisa_null_null_20110217_4_6_1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
URN:NBN:no-nb_digavis_aamliavisa_null_null_20110303_4_8_1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
URN:NBN:no-nb_digavis_aamliavisa_null_null_20110331_4_12_1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
URN:NBN:no-nb_digavis_ytringen_null_null_20151201_34_91_1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
URN:NBN:no-nb_digavis_ytringen_null_null_20160614_35_45_1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
URN:NBN:no-nb_digavis_ytringen_null_null_20160909_35_70_1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Ordfrekvenser totalt i korpuset

Tabellen viser totalfrekvensene av hvert søkeord i korpuset.

In [72]:
word_freqs.sum()

word
barnevern             25568.0
barnevernet           28218.0
barnevernets           1780.0
barnevernloven          452.0
barnevernsnemda           9.0
barneverntjeneste       500.0
barneverntjenesten     1645.0
barneverntjenester      563.0
dtype: float64

## Sentimentscore

Beregn en sentimentscore til kontekstene der nøkkelordet forekommer.
  

In [73]:
# INPUT KREVES
# Fyll inn antall ord som skal telles med i konteksten før og etter nøkkelordet.
before=50
after=50

### FIXME: fjern denne cellen
prosess:
1. **Hent ut kontekster** (kollokasjoner) for nøkkelordet
   - nøkkelord
   - urn --> gir samlet sentimentscore for utgivelsen
   - evt. konkordans (100) per forekomst? --> gir sentimentscore per forekomst.
2. **Beregn sentimentscore** for forekomstene av nøkkelordet
   - kollokasjon
   - positiv ordliste
   - negativ ordliste 

In [74]:

def fetch_finegrained_collocations(urn: str, word: str, before: int, after: int) -> pd.DataFrame:
    """For a given URN and a given word, fetch collocations with """
    coll = urn_collocation(
        urns=[urn],
        word=word,
        before=before,
        after=after
    )
    coll = coll.loc[[x for x in coll.index if x.isalpha()]]
    coll.index = [x.lower() for x in coll.index]
    coll = coll.groupby(coll.index).sum()
    return coll


In [81]:

def score_sentiment(urn, word, before, after):
    """Calculate a sentiment score for the contexts of `word` in a given publication."""
    collocations = fetch_finegrained_collocations(urn, word, before, after)
    pos, neg = coll_sentiment(collocations, word, return_score_only=True)
    return [word, pos, neg, pos-neg]



In [82]:
score_sentiment(news_df.urn[0], search_terms[0], before, after)

['barnevern', 4, 6, -2]

In [ ]:
#corpus_df.apply(score_sentiment, axis=1, result_type="expand", args=(search_term, before, after))


----

In [ ]:
# tester greier 

for urn in corpus_df.urn:
    freq = word_freqs.loc[urn]
    occurring_words = word_freqs.index[word_freqs > 0]
    x = word_freqs[word]
    for word in occurring_words:
        print(f"{word} forekommer {word_freqs.loc[word]} ganger i {corpus_df.title[corpus_df.urn == urn]} {corpus_df["timestamp"][corpus_df.urn == urn] }")

In [ ]:
from dhlab.api.dhlab_api import concordance_counts, concordance

word =occurring_words[1]

conc = concordance(urns=[urn], words=word, window=100, limit=100)
conccount = concordance_counts(urns=[urn], words=word, window=100, limit=100)

In [ ]:

example = conc.conc.values[0]
example


In [ ]:
from dhlab.nbtokenizer import tokenize

tokenize(example)

In [ ]:

coll

##  Lagre data 

**Skriv utdata** til en CSV-fil på ditt lokale filsystem.

Kolonner:
-  URN
-  dato
-  avistittel
-  sted
-  nøkkelord
-  ordfrekvens
-  postiv score
-  negativ score
-  sum sentimentscore


In [ ]:
outputfile = f"data/sentimentanalyse_{search_term}_aviskorpus_{from_year}-{to_year}.csv"
corpus_df.to_csv(outputfile)